In [57]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [58]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
dat=[]
dat2=[]
for n in [3..1000]:
    p=prime(n)
    firsts=[]
    for j in [0..20]: 
        residues=[]
        data=[]
        k=s[j][0]
        poly=D(s[j][1]) # poly(m) = cst term of j_m^power
        for a in [3..5]: # nb
            m=p^a
            polym=evaluate(poly,m)
            o=order(p,polym)
            if a==3:first=o
            test=(a-3)*k+first-o
            dat=dat+[test]
            dat2=dat2+[n]
            print((n,p,k,a,test))
print("union of tests:",uniteList(dat))

(3, 5, 1, 3, 0)
(3, 5, 1, 4, 0)
(3, 5, 1, 5, 0)
(3, 5, 2, 3, 0)
(3, 5, 2, 4, 0)
(3, 5, 2, 5, 0)
(3, 5, 3, 3, 0)
(3, 5, 3, 4, 0)
(3, 5, 3, 5, 0)
(3, 5, 4, 3, 0)
(3, 5, 4, 4, 0)
(3, 5, 4, 5, 0)
(3, 5, 5, 3, 0)
(3, 5, 5, 4, 0)
(3, 5, 5, 5, 0)
(3, 5, 6, 3, 0)
(3, 5, 6, 4, 0)
(3, 5, 6, 5, 0)
(3, 5, 7, 3, 0)
(3, 5, 7, 4, 0)
(3, 5, 7, 5, 0)
(3, 5, 8, 3, 0)
(3, 5, 8, 4, 0)
(3, 5, 8, 5, 0)
(3, 5, 9, 3, 0)
(3, 5, 9, 4, 0)
(3, 5, 9, 5, 0)
(3, 5, 10, 3, 0)
(3, 5, 10, 4, 0)
(3, 5, 10, 5, 0)
(3, 5, 11, 3, 0)
(3, 5, 11, 4, 0)
(3, 5, 11, 5, 0)
(3, 5, 12, 3, 0)
(3, 5, 12, 4, 0)
(3, 5, 12, 5, 0)
(3, 5, 13, 3, 0)
(3, 5, 13, 4, 0)
(3, 5, 13, 5, 0)
(3, 5, 14, 3, 0)
(3, 5, 14, 4, 0)
(3, 5, 14, 5, 0)
(3, 5, 15, 3, 0)
(3, 5, 15, 4, 0)
(3, 5, 15, 5, 0)
(3, 5, 16, 3, 0)
(3, 5, 16, 4, 0)
(3, 5, 16, 5, 0)
(3, 5, 17, 3, 0)
(3, 5, 17, 4, 0)
(3, 5, 17, 5, 0)
(3, 5, 18, 3, 0)
(3, 5, 18, 4, 0)
(3, 5, 18, 5, 0)
(3, 5, 19, 3, 0)
(3, 5, 19, 4, 0)
(3, 5, 19, 5, 0)
(3, 5, 20, 3, 0)
(3, 5, 20, 4, 0)
(3, 5, 20, 5, 0)
(3, 5, 

(11, 31, 18, 4, 0)
(11, 31, 18, 5, 0)
(11, 31, 19, 3, 0)
(11, 31, 19, 4, 0)
(11, 31, 19, 5, 0)
(11, 31, 20, 3, 0)
(11, 31, 20, 4, 0)
(11, 31, 20, 5, 0)
(11, 31, 21, 3, 0)
(11, 31, 21, 4, 0)
(11, 31, 21, 5, 0)
(12, 37, 1, 3, 0)
(12, 37, 1, 4, 0)
(12, 37, 1, 5, 0)
(12, 37, 2, 3, 0)
(12, 37, 2, 4, 0)
(12, 37, 2, 5, 0)
(12, 37, 3, 3, 0)
(12, 37, 3, 4, 0)
(12, 37, 3, 5, 0)
(12, 37, 4, 3, 0)
(12, 37, 4, 4, 0)
(12, 37, 4, 5, 0)
(12, 37, 5, 3, 0)
(12, 37, 5, 4, 0)
(12, 37, 5, 5, 0)
(12, 37, 6, 3, 0)
(12, 37, 6, 4, 0)
(12, 37, 6, 5, 0)
(12, 37, 7, 3, 0)
(12, 37, 7, 4, 0)
(12, 37, 7, 5, 0)
(12, 37, 8, 3, 0)
(12, 37, 8, 4, 0)
(12, 37, 8, 5, 0)
(12, 37, 9, 3, 0)
(12, 37, 9, 4, 0)
(12, 37, 9, 5, 0)
(12, 37, 10, 3, 0)
(12, 37, 10, 4, 0)
(12, 37, 10, 5, 0)
(12, 37, 11, 3, 0)
(12, 37, 11, 4, 0)
(12, 37, 11, 5, 0)
(12, 37, 12, 3, 0)
(12, 37, 12, 4, 0)
(12, 37, 12, 5, 0)
(12, 37, 13, 3, 0)
(12, 37, 13, 4, 0)
(12, 37, 13, 5, 0)
(12, 37, 14, 3, 0)
(12, 37, 14, 4, 0)
(12, 37, 14, 5, 0)
(12, 37, 15, 3, 0)
(

(19, 67, 21, 3, 0)
(19, 67, 21, 4, 0)
(19, 67, 21, 5, 0)
(20, 71, 1, 3, 0)
(20, 71, 1, 4, 0)
(20, 71, 1, 5, 0)
(20, 71, 2, 3, 0)
(20, 71, 2, 4, 0)
(20, 71, 2, 5, 0)
(20, 71, 3, 3, 0)
(20, 71, 3, 4, 0)
(20, 71, 3, 5, 0)
(20, 71, 4, 3, 0)
(20, 71, 4, 4, 0)
(20, 71, 4, 5, 0)
(20, 71, 5, 3, 0)
(20, 71, 5, 4, 0)
(20, 71, 5, 5, 0)
(20, 71, 6, 3, 0)
(20, 71, 6, 4, 0)
(20, 71, 6, 5, 0)
(20, 71, 7, 3, 0)
(20, 71, 7, 4, 0)
(20, 71, 7, 5, 0)
(20, 71, 8, 3, 0)
(20, 71, 8, 4, 0)
(20, 71, 8, 5, 0)
(20, 71, 9, 3, 0)
(20, 71, 9, 4, 0)
(20, 71, 9, 5, 0)
(20, 71, 10, 3, 0)
(20, 71, 10, 4, 0)
(20, 71, 10, 5, 0)
(20, 71, 11, 3, 0)
(20, 71, 11, 4, 0)
(20, 71, 11, 5, 0)
(20, 71, 12, 3, 0)
(20, 71, 12, 4, 0)
(20, 71, 12, 5, 0)
(20, 71, 13, 3, 0)
(20, 71, 13, 4, 0)
(20, 71, 13, 5, 0)
(20, 71, 14, 3, 0)
(20, 71, 14, 4, 0)
(20, 71, 14, 5, 0)
(20, 71, 15, 3, 0)
(20, 71, 15, 4, 0)
(20, 71, 15, 5, 0)
(20, 71, 16, 3, 0)
(20, 71, 16, 4, 0)
(20, 71, 16, 5, 0)
(20, 71, 17, 3, 0)
(20, 71, 17, 4, 0)
(20, 71, 17, 5, 0)
(

(28, 107, 13, 4, 0)
(28, 107, 13, 5, 0)
(28, 107, 14, 3, 0)
(28, 107, 14, 4, 0)
(28, 107, 14, 5, 0)
(28, 107, 15, 3, 0)
(28, 107, 15, 4, 0)
(28, 107, 15, 5, 0)
(28, 107, 16, 3, 0)
(28, 107, 16, 4, 0)
(28, 107, 16, 5, 0)
(28, 107, 17, 3, 0)
(28, 107, 17, 4, 0)
(28, 107, 17, 5, 0)
(28, 107, 18, 3, 0)
(28, 107, 18, 4, 0)
(28, 107, 18, 5, 0)
(28, 107, 19, 3, 0)
(28, 107, 19, 4, 0)
(28, 107, 19, 5, 0)
(28, 107, 20, 3, 0)
(28, 107, 20, 4, 0)
(28, 107, 20, 5, 0)
(28, 107, 21, 3, 0)
(28, 107, 21, 4, 0)
(28, 107, 21, 5, 0)
(29, 109, 1, 3, 0)
(29, 109, 1, 4, 0)
(29, 109, 1, 5, 0)
(29, 109, 2, 3, 0)
(29, 109, 2, 4, 0)
(29, 109, 2, 5, 0)
(29, 109, 3, 3, 0)
(29, 109, 3, 4, 0)
(29, 109, 3, 5, 0)
(29, 109, 4, 3, 0)
(29, 109, 4, 4, 0)
(29, 109, 4, 5, 0)
(29, 109, 5, 3, 0)
(29, 109, 5, 4, 0)
(29, 109, 5, 5, 0)
(29, 109, 6, 3, 0)
(29, 109, 6, 4, 0)
(29, 109, 6, 5, 0)
(29, 109, 7, 3, 0)
(29, 109, 7, 4, 0)
(29, 109, 7, 5, 0)
(29, 109, 8, 3, 0)
(29, 109, 8, 4, 0)
(29, 109, 8, 5, 0)
(29, 109, 9, 3, 0)
(29, 

(36, 151, 18, 4, 0)
(36, 151, 18, 5, 0)
(36, 151, 19, 3, 0)
(36, 151, 19, 4, 0)
(36, 151, 19, 5, 0)
(36, 151, 20, 3, 0)
(36, 151, 20, 4, 0)
(36, 151, 20, 5, 0)
(36, 151, 21, 3, 0)
(36, 151, 21, 4, 0)
(36, 151, 21, 5, 0)
(37, 157, 1, 3, 0)
(37, 157, 1, 4, 0)
(37, 157, 1, 5, 0)
(37, 157, 2, 3, 0)
(37, 157, 2, 4, 0)
(37, 157, 2, 5, 0)
(37, 157, 3, 3, 0)
(37, 157, 3, 4, 0)
(37, 157, 3, 5, 0)
(37, 157, 4, 3, 0)
(37, 157, 4, 4, 0)
(37, 157, 4, 5, 0)
(37, 157, 5, 3, 0)
(37, 157, 5, 4, 0)
(37, 157, 5, 5, 0)
(37, 157, 6, 3, 0)
(37, 157, 6, 4, 0)
(37, 157, 6, 5, 0)
(37, 157, 7, 3, 0)
(37, 157, 7, 4, 0)
(37, 157, 7, 5, 0)
(37, 157, 8, 3, 0)
(37, 157, 8, 4, 0)
(37, 157, 8, 5, 0)
(37, 157, 9, 3, 0)
(37, 157, 9, 4, 0)
(37, 157, 9, 5, 0)
(37, 157, 10, 3, 0)
(37, 157, 10, 4, 0)
(37, 157, 10, 5, 0)
(37, 157, 11, 3, 0)
(37, 157, 11, 4, 0)
(37, 157, 11, 5, 0)
(37, 157, 12, 3, 0)
(37, 157, 12, 4, 0)
(37, 157, 12, 5, 0)
(37, 157, 13, 3, 0)
(37, 157, 13, 4, 0)
(37, 157, 13, 5, 0)
(37, 157, 14, 3, 0)
(37, 15

...

(969, 7639, 6, 3, 0)
(969, 7639, 6, 4, 0)
(969, 7639, 6, 5, 0)
(969, 7639, 7, 3, 0)
(969, 7639, 7, 4, 0)
(969, 7639, 7, 5, 0)
(969, 7639, 8, 3, 0)
(969, 7639, 8, 4, 0)
(969, 7639, 8, 5, 0)
(969, 7639, 9, 3, 0)
(969, 7639, 9, 4, 0)
(969, 7639, 9, 5, 0)
(969, 7639, 10, 3, 0)
(969, 7639, 10, 4, 0)
(969, 7639, 10, 5, 0)
(969, 7639, 11, 3, 0)
(969, 7639, 11, 4, 0)
(969, 7639, 11, 5, 0)
(969, 7639, 12, 3, 0)
(969, 7639, 12, 4, 0)
(969, 7639, 12, 5, 0)
(969, 7639, 13, 3, 0)
(969, 7639, 13, 4, 0)
(969, 7639, 13, 5, 0)
(969, 7639, 14, 3, 0)
(969, 7639, 14, 4, 0)
(969, 7639, 14, 5, 0)
(969, 7639, 15, 3, 0)
(969, 7639, 15, 4, 0)
(969, 7639, 15, 5, 0)
(969, 7639, 16, 3, 0)
(969, 7639, 16, 4, 0)
(969, 7639, 16, 5, 0)
(969, 7639, 17, 3, 0)
(969, 7639, 17, 4, 0)
(969, 7639, 17, 5, 0)
(969, 7639, 18, 3, 0)
(969, 7639, 18, 4, 0)
(969, 7639, 18, 5, 0)
(969, 7639, 19, 3, 0)
(969, 7639, 19, 4, 0)
(969, 7639, 19, 5, 0)
(969, 7639, 20, 3, 0)
(969, 7639, 20, 4, 0)
(969, 7639, 20, 5, 0)
(969, 7639, 21, 3, 0)


(975, 7687, 7, 3, 0)
(975, 7687, 7, 4, 0)
(975, 7687, 7, 5, 0)
(975, 7687, 8, 3, 0)
(975, 7687, 8, 4, 0)
(975, 7687, 8, 5, 0)
(975, 7687, 9, 3, 0)
(975, 7687, 9, 4, 0)
(975, 7687, 9, 5, 0)
(975, 7687, 10, 3, 0)
(975, 7687, 10, 4, 0)
(975, 7687, 10, 5, 0)
(975, 7687, 11, 3, 0)
(975, 7687, 11, 4, 0)
(975, 7687, 11, 5, 0)
(975, 7687, 12, 3, 0)
(975, 7687, 12, 4, 0)
(975, 7687, 12, 5, 0)
(975, 7687, 13, 3, 0)
(975, 7687, 13, 4, 0)
(975, 7687, 13, 5, 0)
(975, 7687, 14, 3, 0)
(975, 7687, 14, 4, 0)
(975, 7687, 14, 5, 0)
(975, 7687, 15, 3, 0)
(975, 7687, 15, 4, 0)
(975, 7687, 15, 5, 0)
(975, 7687, 16, 3, 0)
(975, 7687, 16, 4, 0)
(975, 7687, 16, 5, 0)
(975, 7687, 17, 3, 0)
(975, 7687, 17, 4, 0)
(975, 7687, 17, 5, 0)
(975, 7687, 18, 3, 0)
(975, 7687, 18, 4, 0)
(975, 7687, 18, 5, 0)
(975, 7687, 19, 3, 0)
(975, 7687, 19, 4, 0)
(975, 7687, 19, 5, 0)
(975, 7687, 20, 3, 0)
(975, 7687, 20, 4, 0)
(975, 7687, 20, 5, 0)
(975, 7687, 21, 3, 0)
(975, 7687, 21, 4, 0)
(975, 7687, 21, 5, 0)
(976, 7691, 1, 3, 0

(983, 7753, 3, 5, 0)
(983, 7753, 4, 3, 0)
(983, 7753, 4, 4, 0)
(983, 7753, 4, 5, 0)
(983, 7753, 5, 3, 0)
(983, 7753, 5, 4, 0)
(983, 7753, 5, 5, 0)
(983, 7753, 6, 3, 0)
(983, 7753, 6, 4, 0)
(983, 7753, 6, 5, 0)
(983, 7753, 7, 3, 0)
(983, 7753, 7, 4, 0)
(983, 7753, 7, 5, 0)
(983, 7753, 8, 3, 0)
(983, 7753, 8, 4, 0)
(983, 7753, 8, 5, 0)
(983, 7753, 9, 3, 0)
(983, 7753, 9, 4, 0)
(983, 7753, 9, 5, 0)
(983, 7753, 10, 3, 0)
(983, 7753, 10, 4, 0)
(983, 7753, 10, 5, 0)
(983, 7753, 11, 3, 0)
(983, 7753, 11, 4, 0)
(983, 7753, 11, 5, 0)
(983, 7753, 12, 3, 0)
(983, 7753, 12, 4, 0)
(983, 7753, 12, 5, 0)
(983, 7753, 13, 3, 0)
(983, 7753, 13, 4, 0)
(983, 7753, 13, 5, 0)
(983, 7753, 14, 3, 0)
(983, 7753, 14, 4, 0)
(983, 7753, 14, 5, 0)
(983, 7753, 15, 3, 0)
(983, 7753, 15, 4, 0)
(983, 7753, 15, 5, 0)
(983, 7753, 16, 3, 0)
(983, 7753, 16, 4, 0)
(983, 7753, 16, 5, 0)
(983, 7753, 17, 3, 0)
(983, 7753, 17, 4, 0)
(983, 7753, 17, 5, 0)
(983, 7753, 18, 3, 0)
(983, 7753, 18, 4, 0)
(983, 7753, 18, 5, 0)
(983, 7

(990, 7829, 20, 3, 0)
(990, 7829, 20, 4, 0)
(990, 7829, 20, 5, 0)
(990, 7829, 21, 3, 0)
(990, 7829, 21, 4, 0)
(990, 7829, 21, 5, 0)
(991, 7841, 1, 3, 0)
(991, 7841, 1, 4, 0)
(991, 7841, 1, 5, 0)
(991, 7841, 2, 3, 0)
(991, 7841, 2, 4, 0)
(991, 7841, 2, 5, 0)
(991, 7841, 3, 3, 0)
(991, 7841, 3, 4, 0)
(991, 7841, 3, 5, 0)
(991, 7841, 4, 3, 0)
(991, 7841, 4, 4, 0)
(991, 7841, 4, 5, 0)
(991, 7841, 5, 3, 0)
(991, 7841, 5, 4, 0)
(991, 7841, 5, 5, 0)
(991, 7841, 6, 3, 0)
(991, 7841, 6, 4, 0)
(991, 7841, 6, 5, 0)
(991, 7841, 7, 3, 0)
(991, 7841, 7, 4, 0)
(991, 7841, 7, 5, 0)
(991, 7841, 8, 3, 0)
(991, 7841, 8, 4, 0)
(991, 7841, 8, 5, 0)
(991, 7841, 9, 3, 0)
(991, 7841, 9, 4, 0)
(991, 7841, 9, 5, 0)
(991, 7841, 10, 3, 0)
(991, 7841, 10, 4, 0)
(991, 7841, 10, 5, 0)
(991, 7841, 11, 3, 0)
(991, 7841, 11, 4, 0)
(991, 7841, 11, 5, 0)
(991, 7841, 12, 3, 0)
(991, 7841, 12, 4, 0)
(991, 7841, 12, 5, 0)
(991, 7841, 13, 3, 0)
(991, 7841, 13, 4, 0)
(991, 7841, 13, 5, 0)
(991, 7841, 14, 3, 0)
(991, 7841, 14,

(996, 7879, 21, 5, 0)
(997, 7883, 1, 3, 0)
(997, 7883, 1, 4, 0)
(997, 7883, 1, 5, 0)
(997, 7883, 2, 3, 0)
(997, 7883, 2, 4, 0)
(997, 7883, 2, 5, 0)
(997, 7883, 3, 3, 0)
(997, 7883, 3, 4, 0)
(997, 7883, 3, 5, 0)
(997, 7883, 4, 3, 0)
(997, 7883, 4, 4, 0)
(997, 7883, 4, 5, 0)
(997, 7883, 5, 3, 0)
(997, 7883, 5, 4, 0)
(997, 7883, 5, 5, 0)
(997, 7883, 6, 3, 0)
(997, 7883, 6, 4, 0)
(997, 7883, 6, 5, 0)
(997, 7883, 7, 3, 0)
(997, 7883, 7, 4, 0)
(997, 7883, 7, 5, 0)
(997, 7883, 8, 3, 0)
(997, 7883, 8, 4, 0)
(997, 7883, 8, 5, 0)
(997, 7883, 9, 3, 0)
(997, 7883, 9, 4, 0)
(997, 7883, 9, 5, 0)
(997, 7883, 10, 3, 0)
(997, 7883, 10, 4, 0)
(997, 7883, 10, 5, 0)
(997, 7883, 11, 3, 0)
(997, 7883, 11, 4, 0)
(997, 7883, 11, 5, 0)
(997, 7883, 12, 3, 0)
(997, 7883, 12, 4, 0)
(997, 7883, 12, 5, 0)
(997, 7883, 13, 3, 0)
(997, 7883, 13, 4, 0)
(997, 7883, 13, 5, 0)
(997, 7883, 14, 3, 0)
(997, 7883, 14, 4, 0)
(997, 7883, 14, 5, 0)
(997, 7883, 15, 3, 0)
(997, 7883, 15, 4, 0)
(997, 7883, 15, 5, 0)
(997, 7883, 16,